# NLP Project (Arabic Dialect Classification)-DL-BERT-Base-Arabic

### Importing necessary libraries

In [48]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
import re
!pip install tashaphyne
from tashaphyne.stemming import ArabicLightStemmer
import string
import tashaphyne.arabic_const as arabconst
import string
from nltk.corpus import stopwords
import numpy as np
from keras.losses import CategoricalCrossentropy
#AutoTFBertForSequenceClassification
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
import pandas as pd
from nltk.tokenize import TweetTokenizer
from sklearn.base import BaseEstimator, TransformerMixin


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Load Dataset

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
tweet_df=pd.read_csv('/content/drive/MyDrive/id_text.csv')

In [39]:
tweet_df.head()

,id,text
0,1009754958479151232,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.
1,1009794751548313600,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة
2,1019989115490787200,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند
3,1035479791758135168,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً. ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات؟ يعني اليتيمة متستحقش تتزوج؟ وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل؟ هي متستحقش يندارلها الف حساب ولا هي عبدة؟
4,1035481122921164800,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت. هدي اعصابك وفكينا من التخلف امتاعك


In [40]:
dialect_df=pd.read_csv('/content/drive/MyDrive/id_dialect.csv')

In [41]:
tweet_dialect_df = pd.merge(tweet_df, dialect_df, how='inner', on='id')

In [ ]:
tweet_df

,id,text
0,1009754958479151232,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.
1,1009794751548313600,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة
2,1019989115490787200,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند
3,1035479791758135168,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً. ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات؟ يعني اليتيمة متستحقش تتزوج؟ وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل؟ هي متستحقش يندارلها الف حساب ولا هي عبدة؟
4,1035481122921164800,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت. هدي اعصابك وفكينا من التخلف امتاعك
...,...,...
147720,982775354405376000,الناس دي بتنفخ في قربة مقدودة بالدارجي كده البلد دي لو ما الكيزان ديل راحو مافي حياه شريفه في البلد دي لان سبب الفساد والمصائب الموجودة #التطلع_لسودان_افضل
147721,980728852224860160,@Wail_Hamza @amiromer انت عايش وين بره السودان ولا شنو ماشايف البحصل دا
147722,975502734668500992,مااحرم نفسي ميسي حريف ولعاب برضو ..\nمدريدي وافتخر 🇪🇸
147723,980207908641263616,ياخي ديل ماخلو للشيطان وإبليس شي لكن يروحو وين من ربنا ونحن وراك وراك ياكوز ياعفن الأيام معدوده خلاص #تعرف_علي_الكيزان


In [ ]:
tweet_dialect_df

,id,text,dialect
0,1009754958479151232,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.,LY
1,1009794751548313600,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة,LY
2,1019989115490787200,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند,LY
3,1035479791758135168,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً. ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات؟ يعني اليتيمة متستحقش تتزوج؟ وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل؟ هي متستحقش يندارلها الف حساب ولا هي عبدة؟,LY
4,1035481122921164800,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت. هدي اعصابك وفكينا من التخلف امتاعك,LY
...,...,...,...
147720,982775354405376000,الناس دي بتنفخ في قربة مقدودة بالدارجي كده البلد دي لو ما الكيزان ديل راحو مافي حياه شريفه في البلد دي لان سبب الفساد والمصائب الموجودة #التطلع_لسودان_افضل,SD
147721,980728852224860160,@Wail_Hamza @amiromer انت عايش وين بره السودان ولا شنو ماشايف البحصل دا,SD
147722,975502734668500992,مااحرم نفسي ميسي حريف ولعاب برضو ..\nمدريدي وافتخر 🇪🇸,SD
147723,980207908641263616,ياخي ديل ماخلو للشيطان وإبليس شي لكن يروحو وين من ربنا ونحن وراك وراك ياكوز ياعفن الأيام معدوده خلاص #تعرف_علي_الكيزان,SD


In [42]:
dialect=tweet_dialect_df['dialect'].unique()

In [43]:
dialect

array(['LY', 'MA', 'EG', 'LB', 'SD'], dtype=object)

In [44]:
LY_tweets=tweet_dialect_df[tweet_dialect_df['dialect']=='LY']['text']
MA_tweets=tweet_dialect_df[tweet_dialect_df['dialect']=='MA']['text']
EG_tweets=tweet_dialect_df[tweet_dialect_df['dialect']=='EG']['text']
LB_tweets=tweet_dialect_df[tweet_dialect_df['dialect']=='LB']['text']
SD_tweets=tweet_dialect_df[tweet_dialect_df['dialect']=='SD']['text']


In [45]:
#Count of all tweets
count_LY=len(LY_tweets)/len(tweet_dialect_df)*100
count_MA=len(MA_tweets)/len(tweet_dialect_df)*100
count_EG=len(EG_tweets)/len(tweet_dialect_df)*100
count_LB=len(LB_tweets)/len(tweet_dialect_df)*100
count_SD=len(SD_tweets)/len(tweet_dialect_df)*100

In [46]:
print("count_LY",count_LY)
print("count_MA",count_MA)
print("count_EG",count_EG)
print("count_LB",count_LB)
print("count_SD",count_SD)

count_LY 24.70739549839228
count_MA 7.81113555593163
count_EG 39.01573870367236
count_LB 18.69487222880352
count_SD 9.770858013200204


### Preprocessing

In [50]:

arabic_stopwords = set(stopwords.words('arabic'))

##### Remove stopwords

In [52]:
def remove_arabic_stopwords(text):
    arabic_stop_words= set(stopwords.words('arabic'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in arabic_stop_words]
    return " ".join(filtered_words)

##### remove_non_arabic but preserve arabic hashtags

In [53]:
def remove_non_arabic(text):
    # match any non-Arabic Unicode characters
    arabic_pattern = re.compile(r"[^\u0600-\u06FF\s#]+")
    clean_text = re.sub(arabic_pattern, ' ', text)
    return clean_text

##### Normalize letters

In [54]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

##### Remove_tashkeel

In [55]:
arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [56]:
def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

##### Remove punctuation

In [57]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

In [58]:
def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

##### Remove newlines

In [59]:
def replace_newlines(txt):
    return txt.replace('\n', ' ')

##### Remove reapeated letters

In [60]:
def remove_repeating_char(text):
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    #text = re.sub(r'(.)\1+', r'\1', text)
    text = ' '.join([word for word in text.split() if len(word) > 1])
    return text
    

##### Normalize

In [61]:
def strip_tatweel(text):
 
    return re.sub(u'[%s]' % arabconst.TATWEEL, ' ', text)
def normalize_lamalef(text):

    return arabconst.LAMALEFAT_PAT.sub(\
      u'%s%s'%(arabconst.LAM, arabconst.ALEF), text)

def normalize_searchtext(text):
  
    text = strip_tatweel(text)
    text = normalize_lamalef(text)
   
   
    return text

##### Remove repeated words

In [62]:
def remove_repeated_words(text):
    return  re.sub(r'\b(\w+)(\W+\1)+\b', r'\1', text)
    

##### Remove numbers

In [63]:
def remove_eng_arabic_num(text):
    pattern = r"\d+"
    # Remove numbers from text
    text = re.sub(pattern, "", text)
    return text

##### Cleaning function

In [64]:
def cleaning(df, col_name='text'):
    df[col_name] = df[col_name].apply(remove_arabic_stopwords)
    df[col_name] = df[col_name].apply(remove_non_arabic)
    df[col_name] = df[col_name].apply(normalize_arabic)
    df[col_name] = df[col_name].apply(remove_diacritics)
    df[col_name] = df[col_name].apply(remove_punctuations)
    df[col_name] = df[col_name].apply(replace_newlines)
    df[col_name] = df[col_name].apply(remove_repeating_char)
    df[col_name] = df[col_name].apply(normalize_searchtext)
    df[col_name] = df[col_name].apply(remove_repeated_words)
    df[col_name] = df[col_name].apply(remove_eng_arabic_num)
    return  df

In [65]:
sample_df = tweet_dialect_df.sample(n=50)

In [66]:
cleaning(sample_df, col_name='text')

,id,text,dialect
18384,1156333267047657472,لاباس الصوره مش كويسه حوليها افضل,LY
54785,1101148768198643584,بالليل برضو تخلصي اللي وراكي,EG
59451,1083669319798738944,عاقلين زيي كتير المجانين اغلبيه,EG
144921,1174801136638935040,طيب والله انا مكتءبه وحزينه جدا محد ودهه يخليني ابتسم ويفرحني,SD
113450,1152638125606342784,معقول شي بارجه طالعه عالجنوب,LB
94534,1099253802287489024,قولي هندسه ناقصك ايه المشروع اجبهولك شوال حبي,EG
78367,934806112414650368,بيرنلي عامل منظومه دفاعيه رهيبه الحقيقه يعني,EG
147246,991989175925792768,حلايب مخ البعير جنوبها حمايه الجيش السوداني المصريين شمالها فقط وقريبا نطردهم ونرجعها تيران وصنافير رز,SD
141716,1158727542905430016,بنجي نعتذر ان شاء نجي نلقاك صاحيه,SD
62991,901387233923661824,بقالي شهور مشتركه ولسه وزني زي ماهو مش حاسه باي تغيير شكلها لازم اروح شخصيا عشان اعرف الحكايه,EG


In [31]:
cleaning_tweets=cleaning(tweet_dialect_df, col_name='text')

In [33]:
import numpy as np

import tensorflow as tf

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
texts=cleaning_tweets['text']
labels=cleaning_tweets['dialect']
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [36]:
X_train[0:10].tolist()

['الدنيا دي الحلو والوحش',
 'انا بقيت اعمل كده فكره ليه اوجع ايدي تقليب ممكن اسخر التكنولوجيا لخدمتي',
 'البوست بتاع خافيير سولانا طلع انه هاكر تركي اخترق حسابه لذلك وجب التنويه',
 'احلي دي ايه افريقيا يا وداد',
 'حبيبي قدها كبير والله',
 'بيجن وحبيت كتير البرمو الف مبرووك حبيبي طلتك فخامه وهيبه تءبر قلبي امبين الدور بيشل التطليعه ياويلي الله يعطيك الف عافيه يارب',
 'سمكه اسمها رعيده بترعد بتكهرب يعني حجمها صغير الكل البحر بيخافها لطشتها بروح مش مهم الحجم اهم حاجه الامكانيات',
 'بالنسبه للنجوم اللي بيدافعو المقشفه وبيقولوملكمش حاجه عندهاعشان تحاكموها طاب وبالنسبه للقشف والصدا اللي اتشال بفلوسنا وبالنسبه للعارالمحسوبين عليناقامه ربنايتوب عليو التعريص',
 'اللي هيتابعني هتابعه كلام معلمين الخميس الاميره سااره متابع',
 'مشلفه هيدي اذا صبيه تستعمل هيك منطق وهيك الفاظ بتصور انها محترمه']

In [37]:
train_dataset=pd.concat([X_train,y_train],axis=1)

In [38]:
test_dataset=pd.concat([X_test,y_test],axis=1)

##### Loading the model

In [70]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.5 MB/s eta 0:00:00


In [71]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
train_labels =y_train.tolist()
train_texts =X_train.tolist()

In [45]:
test_labels =y_test.tolist()
test_texts =X_test.tolist()

In [47]:
# Load the pre-trained BERT model and tokenizer
model_name = 'asafaya/bert-base-arabic'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=5)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-base-arabic and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
train_labels =y_train.tolist()
# Convert labels to numerical values
label_dict = {label: index for index, label in enumerate(set(train_labels))}
train_labels = [label_dict[label] for label in train_labels]
train_labels = tf.keras.utils.to_categorical(train_labels)


# Convert the train and validation encodings to NumPy arrays
train_input_ids = np.array(train_encodings['input_ids'])
train_attention_masks = np.array(train_encodings['attention_mask'])
train_labels = np.array(train_labels)

valid_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)
valid_labels = y_test.tolist()

# Convert labels to numerical values
label_dict = {label: index for index, label in enumerate(set(valid_labels))}
valid_labels = [label_dict[label] for label in valid_labels]
valid_labels = tf.keras.utils.to_categorical(valid_labels)

valid_input_ids = np.array(valid_encodings['input_ids'])
valid_attention_masks = np.array(valid_encodings['attention_mask'])
valid_labels = np.array(valid_labels)


##### Training

In [49]:
# Fine-tune the model on the training data
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
loss =tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.CategoricalAccuracy()
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [51]:
history = model.fit([train_input_ids, train_attention_masks], train_labels, epochs=2, batch_size=32, validation_data=([valid_input_ids, valid_attention_masks], valid_labels))

Epoch 1/2
3694/3694 [==============================] - 2786s 734ms/step - loss: 0.6135 - categorical_accuracy: 0.7788 - val_loss: 0.5284 - val_categorical_accuracy: 0.8083
Epoch 2/2
Epoch 2/2
3694/3694 [==============================] - 2707s 733ms/step - loss: 0.4054 - categorical_accuracy: 0.8593 - val_loss: 0.4729 - val_categorical_accuracy: 0.8359
Epoch 1/2
3694/3694 [==============================] - 2701s 731ms/step - loss: 0.2751 - categorical_accuracy: 0.9065 - val_loss: 0.5090 - val_categorical_accuracy: 0.8343
Epoch 2/2
3694/3694 [==============================] - 2703s 732ms/step - loss: 0.1782 - categorical_accuracy: 0.9409 - val_loss: 0.5939 - val_categorical_accuracy: 0.8318


In [80]:
model.save('/content/drive/MyDrive/model/')

### Prediction

In [87]:
class TextCleaner(BaseEstimator, TransformerMixin):
   
    def __init__(self, stop_words=[]):
        self.stop_words = stop_words
        self.tweet_tokenizer = TweetTokenizer()
        self.st = ISRIStemmer()

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.apply(self.remove_arabic_stopwords)
        X = X.apply(self.normalize_arabic)
        X = X.apply(self.remove_non_arabic)
        X = X.apply(self.remove_diacritics)
        X = X.apply(self.remove_punctuations)
        X = X.apply(self.remove_eng_arabic_num)
        X = X.apply(self.remove_repeated_words)
        X = X.apply(self.normalize_searchtext)
        X = X.apply(self.replace_newlines)
        X = X.apply(self.remove_repeating_char)
        return X
    def remove_arabic_stopwords(self, text):
        arabic_stop_words= set(stopwords.words('arabic'))
        words = word_tokenize(text)
        filtered_words = [word for word in words if word not in arabic_stop_words]
        return " ".join(filtered_words)
    def remove_non_arabic(self, text):
        # match any non-Arabic Unicode characters
        arabic_pattern = re.compile(r"[^\u0600-\u06FF\s#]+")
        clean_text = re.sub(arabic_pattern, ' ', text)
        return clean_text
    def normalize_arabic(self, text):
        text = re.sub("[إأآا]", "ا", text)
        text = re.sub("ى", "ي", text)
        text = re.sub("ؤ", "ء", text)
        text = re.sub("ئ", "ء", text)
        text = re.sub("ة", "ه", text)
        text = re.sub("گ", "ك", text)
        return text
    def remove_diacritics(self, text):
        arabic_diacritics = re.compile("""
                                ّ    | # Tashdid
                                َ    | # Fatha
                                ً    | # Tanwin Fath
                                ُ    | # Damma
                                ٌ    | # Tanwin Damm
                                ِ    | # Kasra
                                ٍ    | # Tanwin Kasr
                                ْ    | # Sukun
                                ـ     # Tatwil/Kashida
                            """, re.VERBOSE)
      
        return re.sub(arabic_diacritics, '', text)
    def remove_punctuations(self, text):
        arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
        english_punctuations = string.punctuation
        punctuations_list = arabic_punctuations + english_punctuations
        translator = str.maketrans('', '', punctuations_list)
        return text.translate(translator)
    def replace_newlines(self, text):
        return text.replace('\n', ' ')
    def remove_repeating_char(self, text):
        text = re.sub(r'(.)\1{2,}', r'\1', text)
        #text = re.sub(r'(.)\1+', r'\1', text)
        text = ' '.join([word for word in text.split() if len(word) > 1])
        return text
    def strip_tatweel(self, text):
 
        return re.sub(u'[%s]' % arabconst.TATWEEL, ' ', text)
    def normalize_lamalef(self, text):

        return arabconst.LAMALEFAT_PAT.sub(\
          u'%s%s'%(arabconst.LAM, arabconst.ALEF), text)

    def normalize_searchtext(self, text):
      
        text = strip_tatweel(text)
        text = normalize_lamalef(text)
        return text
    def remove_repeated_words(self, text):
       return  re.sub(r'\b(\w+)(\W+\1)+\b', r'\1', text)
    
    def remove_eng_arabic_num(self, text):
        pattern = r"\d+"
        # Remove numbers from text
        text = re.sub(pattern, "", text)
        return text

     

In [88]:
text= "شنو بيستفيدو الهجوم جامعه مدرسه جامع سوق مدنيين بوابه مليانه سيارات عاءلات يدعسوا مدنيين عاديين اوروبا يفجرو مطار اطفال وبعدين اعترفوا بالتفجير"

In [89]:
import pandas as pd
cleaner = TextCleaner()
cleaned_text = cleaner.transform(pd.Series([text]))

In [91]:
# Load the saved model
model = tf.keras.models.load_model('/content/drive/MyDrive/model')
text= "شنو بيستفيدو الهجوم جامعه مدرسه جامع سوق مدنيين بوابه مليانه سيارات عاءلات يدعسوا مدنيين عاديين اوروبا يفجرو مطار اطفال وبعدين اعترفوا بالتفجير"
def  bert_base_arabic(model,text):
# Create an instance of the TextCleaner pipeline
    cleaner = TextCleaner()
    # Preprocess the text using the TextCleaner pipeline
    clean_text = cleaner.transform(pd.Series([text])).values
    # Load the tokenizer
    tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic')
    inputs = tokenizer.encode_plus(text, padding="max_length", max_length=10, return_tensors="tf")
    model=model
    outputs = model(inputs)
    logits = outputs['logits']
    predictions = tf.argmax(logits, axis=1)
    label_map = {0:'SD', 1 :'MA' , 2:'LB', 3: 'LY', 4:'EG'}
    # Look up the string label using the predicted index label
    predicted_label = label_map[predictions.numpy()[0]]
    return predicted_label


In [92]:
bert_base_arabic(model,text)

'LY'